In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 📉 The IS Curve: Equilibrium in the Goods Market

The **IS (Investment-Saving) curve** represents combinations of the real interest rate ($r$) and the level of output ($Y$) for which the **goods market is in equilibrium**. Equilibrium occurs when planned aggregate expenditure (total spending) equals actual output produced.

This notebook derives and plots the IS curve based on a simple model of aggregate expenditure components.

# 🛒 Components of Aggregate Expenditure (AE)

In a closed economy, planned aggregate expenditure ($AE$) is the sum of consumption ($C$), planned investment ($I$), and government purchases ($G$):

$$AE = C + I + G$$

We assume simple linear functions for consumption and investment:

1.  **Consumption Function:** Consumption depends positively on disposable income ($Y - T$, where $T$ is taxes).
    $$C = c_0 + c_1 (Y - T)$$
    - $c_0$: Autonomous consumption (spending independent of disposable income).
    - $c_1$: Marginal Propensity to Consume (MPC), the fraction of an extra dollar of disposable income that is spent ($0 < c_1 < 1$).
    - $T$: Taxes (assumed lump-sum here).

2.  **Investment Function:** Planned investment depends negatively on the real interest rate ($r$). A higher interest rate increases the cost of borrowing, making fewer investment projects profitable.
    $$I = i_0 - i_1 r$$
    - $i_0$: Autonomous investment (investment independent of the interest rate).
    - $i_1$: Sensitivity of investment to the real interest rate ($i_1 > 0$).

3.  **Government Purchases ($G$) and Taxes ($T$):** Assumed exogenous (determined outside the model).

# ⚖️ Goods Market Equilibrium & IS Curve Derivation

Equilibrium in the goods market requires that **actual output (Y) equals planned aggregate expenditure (AE)**:

$$Y = AE$$
$$Y = C + I + G$$

Substituting the behavioral equations for $C$ and $I$:

$$Y = [c_0 + c_1 (Y - T)] + [i_0 - i_1 r] + G$$

Now, we solve this equation for $Y$ to find the equilibrium output level for any given interest rate $r$. This relationship *is* the IS curve.

$$Y = c_0 + c_1 Y - c_1 T + i_0 - i_1 r + G$$
$$Y - c_1 Y = c_0 - c_1 T + i_0 + G - i_1 r$$
$$Y (1 - c_1) = (c_0 - c_1 T + i_0 + G) - i_1 r$$

Let $A_0 = c_0 - c_1 T + i_0 + G$ represent the **autonomous expenditure** (spending that doesn't depend on $Y$ or $r$). The term $(1 - c_1)$ is the Marginal Propensity to Save (MPS).

$$Y = \frac{1}{1 - c_1} (A_0 - i_1 r)$$

This is the equation for the **IS Curve**. It shows a **negative relationship** between the real interest rate ($r$) and the equilibrium level of output ($Y$).

* **Slope:** The IS curve slopes downward because a higher $r$ reduces investment ($I$), which lowers aggregate expenditure ($AE$) and thus reduces the equilibrium level of output ($Y$). The steepness depends on the MPC ($c_1$) and the interest sensitivity of investment ($i_1$). A larger $c_1$ or a larger $i_1$ makes the IS curve flatter.
* **Shifts:** The IS curve shifts to the right if autonomous expenditure ($A_0$) increases (due to higher $c_0$, $i_0$, $G$, or lower $T$). It shifts left if $A_0$ decreases.

The simulation below plots this IS curve and allows you to see how changes in the underlying parameters ($c_0, c_1, i_0, i_1, G, T$) affect its position and slope. You can also overlay shifted curves to visualize the impact of policy changes or shocks.

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown, Layout
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def plot_is_curve(c0=20.0, c1=0.7, i0=25.0, i1=50.0, G=50.0, T_tax=30.0, overlay='None'):
    """
    Calculates and plots the IS curve based on aggregate expenditure components.

    Args:
        c0 (float): Autonomous consumption.
        c1 (float): Marginal Propensity to Consume (MPC).
        i0 (float): Autonomous investment.
        i1 (float): Interest sensitivity of investment.
        G (float): Government purchases.
        T_tax (float): Lump-sum taxes.
        overlay (str): Type of policy shift to overlay ('None', 'Higher G',
                       'Lower T', 'Investment Shock').
    """
    # Input validation
    c1 = np.clip(c1, 0.01, 0.99) # MPC between 0 and 1
    i1 = max(i1, 1e-6) # Ensure sensitivity is positive

    # Define range for real interest rate (r)
    r_vals = np.linspace(0.0, 0.25, 200) # Range from 0% to 25%

    # Calculate Autonomous Expenditure (A0)
    A0 = c0 - c1 * T_tax + i0 + G

    # Calculate Multiplier (Keynesian multiplier)
    multiplier = 1.0 / (1.0 - c1)

    # Calculate IS curve: Y = multiplier * (A0 - i1*r)
    Y_vals = multiplier * (A0 - i1 * r_vals)

    # --- Plotting ---
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot the baseline IS curve
    ax.plot(Y_vals, r_vals * 100, label='IS Curve', color='blue', linewidth=2.5) # Plot r on y-axis (conventional)

    # --- Optional Overlays for Shifts ---
    label_shift = ""
    # Store original parameters for comparison text
    params_orig = {'c0': c0, 'c1': c1, 'i0': i0, 'i1': i1, 'G': G, 'T_tax': T_tax}
    params_shift = params_orig.copy()

    if overlay == 'Higher G':
        G_shift = G + 20
        params_shift['G'] = G_shift
        A0_shift = c0 - c1 * T_tax + i0 + G_shift
        Y_shift = multiplier * (A0_shift - i1 * r_vals)
        ax.plot(Y_shift, r_vals * 100, label=f'IS\' (G = {G_shift:.0f})', linestyle='--', color='green', linewidth=2)
        label_shift = f"Higher G (+{G_shift-G:.0f})"
    elif overlay == 'Lower T':
        T_shift = T_tax - 15
        params_shift['T_tax'] = T_shift
        A0_shift = c0 - c1 * T_shift + i0 + G
        Y_shift = multiplier * (A0_shift - i1 * r_vals)
        ax.plot(Y_shift, r_vals * 100, label=f'IS\' (T = {T_shift:.0f})', linestyle='--', color='purple', linewidth=2)
        label_shift = f"Lower T (-{T_tax-T_shift:.0f})"
    elif overlay == 'Investment Shock': # Negative shock to autonomous investment
        i0_shift = i0 - 15
        params_shift['i0'] = i0_shift
        A0_shift = c0 - c1 * T_tax + i0_shift + G
        Y_shift = multiplier * (A0_shift - i1 * r_vals)
        ax.plot(Y_shift, r_vals * 100, label=f'IS\' (i0 = {i0_shift:.0f})', linestyle='--', color='red', linewidth=2)
        label_shift = f"Lower i0 (-{i0-i0_shift:.0f})"

    # Annotate an example point (e.g., at r = 5%)
    r_example = 0.05
    Y_example = multiplier * (A0 - i1 * r_example)
    ax.scatter(Y_example, r_example * 100, color='black', s=80, zorder=5, label=f'Point at r={r_example:.0%}')
    ax.annotate(f" Y = {Y_example:.1f}\n @ r = {r_example:.0%}", xy=(Y_example, r_example * 100),
                xytext=(15, -15), textcoords='offset points',
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=-0.2'),
                fontsize=10, bbox=dict(boxstyle='round,pad=0.3', fc='white', alpha=0.7))

    # Add annotation for shift if overlay is active
    if overlay != 'None':
         # Find a point on the shifted curve to annotate
         Y_example_shift = multiplier * (A0_shift - i1 * r_example)
         ax.scatter(Y_example_shift, r_example * 100, color='grey', s=60, zorder=4)
         ax.annotate(f"{label_shift}\nY = {Y_example_shift:.1f}", xy=(Y_example_shift, r_example * 100),
                     xytext=(-30, 30), textcoords='offset points', ha='right',
                     arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.3', color='grey'),
                     fontsize=9, bbox=dict(boxstyle='round,pad=0.3', fc='lightgrey', alpha=0.7))


    ax.set_title("IS Curve: Goods Market Equilibrium (Y vs r)")
    ax.set_xlabel("Output (Y)")
    ax.set_ylabel("Real Interest Rate (r %)")
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1f}%')) # Format r as percentage

    # Set reasonable plot limits, ensuring origin is visible if appropriate
    ax.set_xlim(left=max(0, Y_vals[r_vals<=0.2].min()*0.8) if Y_vals[r_vals<=0.2].min() > 0 else Y_vals[r_vals<=0.2].min()*1.1)
    ax.set_ylim(bottom=-1) # Start slightly below 0%

    plt.tight_layout()
    plt.show()

    # --- Display IS Equation and Multiplier ---
    is_equation = f"$Y = \\frac{{1}}{{1 - {c1:.2f}}} \\times ({A0:.1f} - {i1:.1f} r)$"
    is_simplified = f"$Y = {multiplier:.2f} \\times ({A0:.1f} - {i1:.1f} r)$"

    results_md = f"""
    ### ⚙️ IS Curve Details:

    * **Autonomous Expenditure (A₀):** ${A0:.2f}$
        * $(A₀ = c₀ - c₁T + i₀ + G)$
    * **Multiplier (1 / (1 - c₁)):** ${multiplier:.2f}$
    * **IS Curve Equation:** {is_equation}
    * **Simplified:** {is_simplified}
    * **Output at r = 5%:** ${Y_example:.2f}$
    """
    if overlay != 'None':
        Y_example_shift = multiplier * (A0_shift - i1 * r_example)
        results_md += f"* **Shifted Output at r = 5% ({label_shift}):** ${Y_example_shift:.2f}$\n"

    display(Markdown(results_md))


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'}
layout = Layout(width='95%')

interact(
    plot_is_curve,
    c0=FloatSlider(value=20.0, min=0, max=50, step=1, description='Autonomous C (c0):', style=style, layout=layout),
    c1=FloatSlider(value=0.7, min=0.1, max=0.95, step=0.05, description='MPC (c1):', style=style, layout=layout, readout_format='.2f'),
    i0=FloatSlider(value=25.0, min=0, max=50, step=1, description='Autonomous I (i0):', style=style, layout=layout),
    i1=FloatSlider(value=50.0, min=10, max=200, step=5, description='Interest Sensitivity (i1):', style=style, layout=layout),
    G=FloatSlider(value=50.0, min=0, max=100, step=5, description='Govt Spending (G):', style=style, layout=layout),
    T_tax=FloatSlider(value=30.0, min=0, max=100, step=5, description='Taxes (T):', style=style, layout=layout),
    overlay=Dropdown(options=['None', 'Higher G', 'Lower T', 'Investment Shock'], value='None', description='Overlay Shift:', style=style, layout=layout),
);


interactive(children=(FloatSlider(value=20.0, description='Autonomous C (c0):', layout=Layout(width='95%'), ma…